## LangGraph Open Deep Research - Supervisor-Researcher Architecture

In this notebook, we'll explore the **supervisor-researcher delegation architecture** for conducting deep research with LangGraph.

You can visit this repository to see the original application: [Open Deep Research](https://github.com/langchain-ai/open_deep_research)

Let's jump in!

## What We're Building

This implementation uses a **hierarchical delegation pattern** where:

1. **User Clarification** - Optionally asks clarifying questions to understand the research scope
2. **Research Brief Generation** - Transforms user messages into a structured research brief
3. **Supervisor** - A lead researcher that analyzes the brief and delegates research tasks
4. **Parallel Researchers** - Multiple sub-agents that conduct focused research simultaneously
5. **Research Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings are combined into a comprehensive report

![Architecture Diagram](https://i.imgur.com/Q8HEZn0.png)

This differs from a section-based approach by allowing dynamic task decomposition based on the research question, rather than predefined sections.

---

# 🤝 Breakout Room #1
## Deep Research Foundations

In this breakout room, we'll understand the architecture and components of the Open Deep Research system.

## Task 1: Dependencies

You'll need API keys for Anthropic (for the LLM) and Tavily (for web search). We'll configure the system to use Anthropic's Claude Sonnet 4 exclusively.

In [1]:
import os
import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

In [28]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

## Task 2: State Definitions

The state structure is hierarchical with three levels:

### Agent State (Top Level)
Contains the overall conversation messages, research brief, accumulated notes, and final report.

### Supervisor State (Middle Level)
Manages the research supervisor's messages, research iterations, and coordinating parallel researchers.

### Researcher State (Bottom Level)
Each individual researcher has their own message history, tool call iterations, and research findings.

We also have structured outputs for tool calling:
- **ConductResearch** - Tool for supervisor to delegate research to a sub-agent
- **ResearchComplete** - Tool to signal research phase is done
- **ClarifyWithUser** - Structured output for asking clarifying questions
- **ResearchQuestion** - Structured output for the research brief

Let's import these from our library: [`open_deep_library/state.py`](open_deep_library/state.py)

In [2]:
# Import state definitions from the library
from open_deep_library.state import (
    # Main workflow states
    AgentState,           # Lines 65-72: Top-level agent state with messages, research_brief, notes, final_report
    AgentInputState,      # Lines 62-63: Input state is just messages
    
    # Supervisor states
    SupervisorState,      # Lines 74-81: Supervisor manages research delegation and iterations
    
    # Researcher states
    ResearcherState,      # Lines 83-90: Individual researcher with messages and tool iterations
    ResearcherOutputState, # Lines 92-96: Output from researcher (compressed research + raw notes)
    
    # Structured outputs for tool calling
    ConductResearch,      # Lines 15-19: Tool for delegating research to sub-agents
    ResearchComplete,     # Lines 21-22: Tool to signal research completion
    ClarifyWithUser,      # Lines 30-41: Structured output for user clarification
    ResearchQuestion,     # Lines 43-48: Structured output for research brief
)

## Task 3: Utility Functions and Tools

The system uses several key utilities:

### Search Tools
- **tavily_search** - Async web search with automatic summarization to stay within token limits
- Supports Anthropic native web search and Tavily API

### Reflection Tools
- **think_tool** - Allows researchers to reflect on their progress and plan next steps (ReAct pattern)

### Helper Utilities
- **get_all_tools** - Assembles the complete toolkit (search + MCP + reflection)
- **get_today_str** - Provides current date context for research
- Token limit handling utilities for graceful degradation

These are defined in [`open_deep_library/utils.py`](open_deep_library/utils.py)

In [3]:
# Import utility functions and tools from the library
from open_deep_library.utils import (
    # Search tool - Lines 43-136: Tavily search with automatic summarization
    tavily_search,
    
    # Reflection tool - Lines 219-244: Strategic thinking tool for ReAct pattern
    think_tool,
    
    # Tool assembly - Lines 569-597: Get all configured tools
    get_all_tools,
    
    # Date utility - Lines 872-879: Get formatted current date
    get_today_str,
    
    # Supporting utilities for error handling
    get_api_key_for_model,          # Lines 892-914: Get API keys from config or env
    is_token_limit_exceeded,         # Lines 665-701: Detect token limit errors
    get_model_token_limit,           # Lines 831-846: Look up model's token limit
    remove_up_to_last_ai_message,    # Lines 848-866: Truncate messages for retry
    anthropic_websearch_called,      # Lines 607-637: Detect Anthropic native search usage
    openai_websearch_called,         # Lines 639-658: Detect OpenAI native search usage
    get_notes_from_tool_calls,       # Lines 599-601: Extract notes from tool messages
)

## Task 4: Configuration System

The configuration system controls:

### Research Behavior
- **allow_clarification** - Whether to ask clarifying questions before research
- **max_concurrent_research_units** - How many parallel researchers can run (default: 5)
- **max_researcher_iterations** - How many times supervisor can delegate research (default: 6)
- **max_react_tool_calls** - Tool call limit per researcher (default: 10)

### Model Configuration
- **research_model** - Model for research and supervision (we'll use Anthropic)
- **compression_model** - Model for synthesizing findings
- **final_report_model** - Model for writing the final report
- **summarization_model** - Model for summarizing web search results

### Search Configuration
- **search_api** - Which search API to use (ANTHROPIC, TAVILY, or NONE)
- **max_content_length** - Character limit before summarization

Defined in [`open_deep_library/configuration.py`](open_deep_library/configuration.py)

In [4]:
# Import configuration from the library
from open_deep_library.configuration import (
    Configuration,    # Lines 38-247: Main configuration class with all settings
    SearchAPI,        # Lines 11-17: Enum for search API options (ANTHROPIC, TAVILY, NONE)
)

## Task 5: Prompt Templates

The system uses carefully engineered prompts for each phase:

### Phase 1: Clarification
**clarify_with_user_instructions** - Analyzes if the research scope is clear or needs clarification

### Phase 2: Research Brief
**transform_messages_into_research_topic_prompt** - Converts user messages into a detailed research brief

### Phase 3: Supervisor
**lead_researcher_prompt** - System prompt for the supervisor that manages delegation strategy

### Phase 4: Researcher
**research_system_prompt** - System prompt for individual researchers conducting focused research

### Phase 5: Compression
**compress_research_system_prompt** - Prompt for synthesizing research findings without losing information

### Phase 6: Final Report
**final_report_generation_prompt** - Comprehensive prompt for writing the final report

All prompts are defined in [`open_deep_library/prompts.py`](open_deep_library/prompts.py)

In [5]:
# Import prompt templates from the library
from open_deep_library.prompts import (
    clarify_with_user_instructions,                    # Lines 3-41: Ask clarifying questions
    transform_messages_into_research_topic_prompt,     # Lines 44-77: Generate research brief
    lead_researcher_prompt,                            # Lines 79-136: Supervisor system prompt
    research_system_prompt,                            # Lines 138-183: Researcher system prompt
    compress_research_system_prompt,                   # Lines 186-222: Research compression prompt
    final_report_generation_prompt,                    # Lines 228-308: Final report generation
)

## ❓ Question #1:

Explain the interrelationships between the three states (Agent, Supervisor, Researcher). Why don't we just make a single huge state?

##### Answer:
The three states split responsibilities by scope: the Agent State keeps the big picture and final answer, the Supervisor State interprets that goal, handles planning and coordination, and each Researcher State focuses on one task in isolation. Information flows down to Researcher to execute work and back up to the Agent to combine results into a final report. Researchers only interact with the supervisor, not with each other or the Agent state. This setup makes parallel research easier and avoids researchers interfering with each other.
A single huge state would mix planning, research, and synthesis into one messy context. That leads to faster context bloat and more brittle behavior. The hierarchical structure keeps things cleaner, cheaper, and easier to reason about.

## ❓ Question #2:

What are the advantages and disadvantages of importing these components instead of including them in the notebook?

##### Answer:
The advantages are that importing everything from separate Python files keeps the notebook clean and readable, so we can focus on running and inspecting results instead of scrolling through setup code. It makes things more modular and reusable, works much better with Git, and feels closer to how we'd structure real production code.
The disadvantages are that there’s a bit more overhead with imports, paths, and reloads, and we often have to reload modules or restart the kernel to see changes. Debugging also feels less instant than just editing cells directly, especially when we’re tweaking prompts or small bits of logic. For quick experiments, that extra layer can slow things down a little.

## 🏗️ Activity #1: Explore the Prompts

Open `open_deep_library/prompts.py` and examine one of the prompt templates in detail.

**Requirements:**
1. Choose one prompt template (clarify, brief, supervisor, researcher, compression, or final report)
2. Explain what the prompt is designed to accomplish
3. Identify 2-3 key techniques used in the prompt (e.g., structured output, role definition, examples)
4. Suggest one improvement you might make to the prompt

**YOUR CODE HERE** - Write your analysis in a markdown cell below

The supervisor (lead_researcher_prompt) prompt template defines the research supervisor whose sole responsibility is to manage research rather than perform it directly. It plans its approach using the think_tool, delegates tasks to researchers through the ConductResearch tool, and then reflects on the results again with think_tool before deciding on next steps. This creates a deliberate loop of planning, delegating, assessing, repeating, and eventually terminating once enough information has been gathered. Finally, it calls ResearchComplete to indicate that the research is complete.

First, it uses clear role definition. By explicitly stating “You are a research supervisor,” the prompt narrowly defines the agent’s responsibility around coordination and decision-making, rather than direct research or answer writing. This immediately sets expectations and keeps the agent focused on managing the process.
Second, the prompt provides concrete examples for how to scale sub-agents based on task type. For simple fact-finding tasks, lists, or rankings, it recommends using a single sub-agent, as these requests do not usually benefit from parallelization. In contrast, when the user asks for comparisons, the prompt suggests assigning one sub-agent per element being compared—for example, using three agents when comparing OpenAI, Anthropic, and DeepMind. These examples act as practical heuristics, helping the supervisor quickly decide how to decompose a task without overthinking.
Third, the prompt enforces hard constraints to prevent excessive delegation. It introduces explicit limits such as a bias toward using a single agent by default, stopping once there is enough information to answer confidently, and capping the total number of tool calls. It also restricts the maximum number of parallel agents per iteration. Together, these constraints serve as guardrails that discourage perfectionism and uncontrolled expansion, ensuring the research process remains efficient and resource-aware.

One improvement I’d suggest is adding a clear research-quality check before calling ResearchComplete. Right now, the decision to stop is mostly based on a general feeling of being “confident,” which can be pretty subjective. This could be made more reliable with a simple checklist, like making sure all major parts of the user’s question are covered, confirming that important claims are backed by at least two independent sources or perspectives, and clearly noting any remaining gaps or uncertainties. Having this kind of quality gate in place would help prevent stopping too early, push in some quick reality checks on what’s been covered, and make the whole research process feel more consistent, especially when the questions get complex.

---

# 🤝 Breakout Room #2
## Building & Running the Researcher

In this breakout room, we'll explore the node functions, build the graph, and run wellness research.

## Task 6: Node Functions - The Building Blocks

Now let's look at the node functions that make up our graph. We'll import them from the library and understand what each does.

### The Complete Research Workflow

The workflow consists of 8 key nodes organized into 3 subgraphs:

1. **Main Graph Nodes:**
   - `clarify_with_user` - Entry point that checks if clarification is needed
   - `write_research_brief` - Transforms user input into structured research brief
   - `final_report_generation` - Synthesizes all research into final report

2. **Supervisor Subgraph Nodes:**
   - `supervisor` - Lead researcher that plans and delegates
   - `supervisor_tools` - Executes supervisor's tool calls (delegation, reflection)

3. **Researcher Subgraph Nodes:**
   - `researcher` - Individual researcher conducting focused research
   - `researcher_tools` - Executes researcher's tool calls (search, reflection)
   - `compress_research` - Synthesizes researcher's findings

All nodes are defined in [`open_deep_library/deep_researcher.py`](open_deep_library/deep_researcher.py)

### Node 1: clarify_with_user

**Purpose:** Analyzes user messages and asks clarifying questions if the research scope is unclear.

**Key Steps:**
1. Check if clarification is enabled in configuration
2. Use structured output to analyze if clarification is needed
3. If needed, end with a clarifying question for the user
4. If not needed, proceed to research brief with verification message

**Implementation:** [`open_deep_library/deep_researcher.py` lines 60-115](open_deep_library/deep_researcher.py#L60-L115)

In [6]:
# Import the clarify_with_user node
from open_deep_library.deep_researcher import clarify_with_user

### Node 2: write_research_brief

**Purpose:** Transforms user messages into a structured research brief for the supervisor.

**Key Steps:**
1. Use structured output to generate detailed research brief from messages
2. Initialize supervisor with system prompt and research brief
3. Set up supervisor messages with proper context

**Why this matters:** A well-structured research brief helps the supervisor make better delegation decisions.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 118-175](open_deep_library/deep_researcher.py#L118-L175)

In [7]:
# Import the write_research_brief node
from open_deep_library.deep_researcher import write_research_brief

### Node 3: supervisor

**Purpose:** Lead research supervisor that plans research strategy and delegates to sub-researchers.

**Key Steps:**
1. Configure model with three tools:
   - `ConductResearch` - Delegate research to a sub-agent
   - `ResearchComplete` - Signal that research is done
   - `think_tool` - Strategic reflection before decisions
2. Generate response based on current context
3. Increment research iteration count
4. Proceed to tool execution

**Decision Making:** The supervisor uses `think_tool` to reflect before delegating research, ensuring thoughtful decomposition of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 178-223](open_deep_library/deep_researcher.py#L178-L223)

In [8]:
# Import the supervisor node (from supervisor subgraph)
from open_deep_library.deep_researcher import supervisor

### Node 4: supervisor_tools

**Purpose:** Executes the supervisor's tool calls, including strategic thinking and research delegation.

**Key Steps:**
1. Check exit conditions:
   - Exceeded maximum iterations
   - No tool calls made
   - `ResearchComplete` called
2. Process `think_tool` calls for strategic reflection
3. Execute `ConductResearch` calls in parallel:
   - Spawn researcher subgraphs for each delegation
   - Limit to `max_concurrent_research_units` (default: 5)
   - Gather all results asynchronously
4. Aggregate findings and return to supervisor

**Parallel Execution:** This is where the magic happens - multiple researchers work simultaneously on different aspects of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 225-349](open_deep_library/deep_researcher.py#L225-L349)

In [9]:
# Import the supervisor_tools node
from open_deep_library.deep_researcher import supervisor_tools

### Node 5: researcher

**Purpose:** Individual researcher that conducts focused research on a specific topic.

**Key Steps:**
1. Load all available tools (search, MCP, reflection)
2. Configure model with tools and researcher system prompt
3. Generate response with tool calls
4. Increment tool call iteration count

**ReAct Pattern:** Researchers use `think_tool` to reflect after each search, deciding whether to continue or provide their answer.

**Available Tools:**
- Search tools (Tavily or Anthropic native search)
- `think_tool` for strategic reflection
- `ResearchComplete` to signal completion
- MCP tools (if configured)

**Implementation:** [`open_deep_library/deep_researcher.py` lines 365-424](open_deep_library/deep_researcher.py#L365-L424)

In [10]:
# Import the researcher node (from researcher subgraph)
from open_deep_library.deep_researcher import researcher

### Node 6: researcher_tools

**Purpose:** Executes the researcher's tool calls, including searches and strategic reflection.

**Key Steps:**
1. Check early exit conditions (no tool calls, native search used)
2. Execute all tool calls in parallel:
   - Search tools fetch and summarize web content
   - `think_tool` records strategic reflections
   - MCP tools execute external integrations
3. Check late exit conditions:
   - Exceeded `max_react_tool_calls` (default: 10)
   - `ResearchComplete` called
4. Continue research loop or proceed to compression

**Error Handling:** Safely handles tool execution errors and continues with available results.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 435-509](open_deep_library/deep_researcher.py#L435-L509)

In [11]:
# Import the researcher_tools node
from open_deep_library.deep_researcher import researcher_tools

### Node 7: compress_research

**Purpose:** Compresses and synthesizes research findings into a concise, structured summary.

**Key Steps:**
1. Configure compression model
2. Add compression instruction to messages
3. Attempt compression with retry logic:
   - If token limit exceeded, remove older messages
   - Retry up to 3 times
4. Extract raw notes from tool and AI messages
5. Return compressed research and raw notes

**Why Compression?** Researchers may accumulate lots of tool outputs and reflections. Compression ensures:
- All important information is preserved
- Redundant information is deduplicated
- Content stays within token limits for the final report

**Token Limit Handling:** Gracefully handles token limit errors by progressively truncating messages.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 511-585](open_deep_library/deep_researcher.py#L511-L585)

In [12]:
# Import the compress_research node
from open_deep_library.deep_researcher import compress_research

### Node 8: final_report_generation

**Purpose:** Generates the final comprehensive research report from all collected findings.

**Key Steps:**
1. Extract all notes from completed research
2. Configure final report model
3. Attempt report generation with retry logic:
   - If token limit exceeded, truncate findings by 10%
   - Retry up to 3 times
4. Return final report or error message

**Token Limit Strategy:**
- First retry: Use model's token limit × 4 as character limit
- Subsequent retries: Reduce by 10% each time
- Graceful degradation with helpful error messages

**Report Quality:** The prompt guides the model to create well-structured reports with:
- Proper headings and sections
- Inline citations
- Comprehensive coverage of all findings
- Sources section at the end

**Implementation:** [`open_deep_library/deep_researcher.py` lines 607-697](open_deep_library/deep_researcher.py#L607-L697)

In [13]:
# Import the final_report_generation node
from open_deep_library.deep_researcher import final_report_generation

## Task 7: Graph Construction - Putting It All Together

The system is organized into three interconnected graphs:

### 1. Researcher Subgraph (Bottom Level)
Handles individual focused research on a specific topic:
```
START → researcher → researcher_tools → compress_research → END
               ↑            ↓
               └────────────┘ (loops until max iterations or ResearchComplete)
```

### 2. Supervisor Subgraph (Middle Level)
Manages research delegation and coordination:
```
START → supervisor → supervisor_tools → END
            ↑              ↓
            └──────────────┘ (loops until max iterations or ResearchComplete)
            
supervisor_tools spawns multiple researcher_subgraphs in parallel
```

### 3. Main Deep Researcher Graph (Top Level)
Orchestrates the complete research workflow:
```
START → clarify_with_user → write_research_brief → research_supervisor → final_report_generation → END
                 ↓                                       (supervisor_subgraph)
               (may end early if clarification needed)
```

Let's import the compiled graphs from the library.

In [14]:
# Import the pre-compiled graphs from the library
from open_deep_library.deep_researcher import (
    # Bottom level: Individual researcher workflow
    researcher_subgraph,    # Lines 588-605: researcher → researcher_tools → compress_research
    
    # Middle level: Supervisor coordination
    supervisor_subgraph,    # Lines 351-363: supervisor → supervisor_tools (spawns researchers)
    
    # Top level: Complete research workflow
    deep_researcher,        # Lines 699-719: Main graph with all phases
)

## Why This Architecture?

### Advantages of Supervisor-Researcher Delegation

1. **Dynamic Task Decomposition**
   - Unlike section-based approaches with predefined structure, the supervisor can break down research based on the actual question
   - Adapts to different types of research (comparisons, lists, deep dives, etc.)

2. **Parallel Execution**
   - Multiple researchers work simultaneously on different aspects
   - Much faster than sequential section processing
   - Configurable parallelism (1-20 concurrent researchers)

3. **ReAct Pattern for Quality**
   - Researchers use `think_tool` to reflect after each search
   - Prevents excessive searching and improves search quality
   - Natural stopping conditions based on information sufficiency

4. **Flexible Tool Integration**
   - Easy to add MCP tools for specialized research
   - Supports multiple search APIs (Anthropic, Tavily)
   - Each researcher can use different tool combinations

5. **Graceful Token Limit Handling**
   - Compression prevents token overflow
   - Progressive truncation in final report generation
   - Research can scale to arbitrary depths

### Trade-offs

- **Complexity:** More moving parts than section-based approach
- **Cost:** Parallel researchers use more tokens (but faster)
- **Unpredictability:** Research structure emerges dynamically

## Task 8: Running the Deep Researcher

Now let's see the system in action! We'll use it to research wellness strategies for improving sleep quality.

### Setup

We need to:
1. Set up the wellness research request
2. Configure the execution with Anthropic settings
3. Run the research workflow

In [15]:
# Set up the graph with Anthropic configuration
from IPython.display import Markdown, display
import uuid

# Note: deep_researcher is already compiled from the library
# For this demo, we'll use it directly without additional checkpointing
graph = deep_researcher

print("✓ Graph ready for execution")
print("  (Note: The graph is pre-compiled from the library)")

✓ Graph ready for execution
  (Note: The graph is pre-compiled from the library)


### Configuration for Anthropic

We'll configure the system to use:
- **Claude Sonnet 4** for all research, supervision, and report generation
- **Tavily** for web search (you can also use Anthropic's native search)
- **Moderate parallelism** (1 concurrent researcher for cost control)
- **Clarification enabled** (will ask if research scope is unclear)

In [16]:
# Configure for Anthropic with moderate settings
config = {
    "configurable": {
        # Model configuration - using Claude Sonnet 4 for everything
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        
        # Research behavior
        "allow_clarification": True,
        "max_concurrent_research_units": 1,  # 1 parallel researcher
        "max_researcher_iterations": 2,      # Supervisor can delegate up to 2 times
        "max_react_tool_calls": 3,           # Each researcher can make up to 3 tool calls
        
        # Search configuration
        "search_api": "tavily",  # Using Tavily for web search
        "max_content_length": 50000,
        
        # Thread ID for this conversation
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: Claude Sonnet 4")
print(f"  - Max Concurrent Researchers: 1")
print(f"  - Max Iterations: 2")
print(f"  - Search API: Tavily")

✓ Configuration ready
  - Research Model: Claude Sonnet 4
  - Max Concurrent Researchers: 1
  - Max Iterations: 2
  - Search API: Tavily


### Execute the Wellness Research

Now let's run the research! We'll ask the system to research evidence-based strategies for improving sleep quality.

The workflow will:
1. **Clarify** - Check if the request is clear (may skip if obvious)
2. **Research Brief** - Transform our request into a structured brief
3. **Supervisor** - Plan research strategy and delegate to researchers
4. **Parallel Research** - Researchers gather information simultaneously
5. **Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings combined into comprehensive report

In [18]:
# Create our wellness research request
research_request = """
I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please research the best evidence-based strategies for improving sleep quality and create a comprehensive sleep improvement plan for me.
"""

# Execute the graph
async def run_research():
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with your sleep improvement research request. I understand you're looking for evidence-based strategies to address your current sleep challenges, which include inconsistent bedtimes (10pm-1am), phone use in bed, and morning fatigue. I will now research comprehensive, science-backed sleep improvement strategies and create a personalized plan that addresses your specific issues while providing actionable recommendations for better sleep quality.

Node: write_research_brief

Research Brief Generated:
I need comprehensive, evidence-based strategies to improve my sleep quality and create a personalized sleep improvement plan. My current sleep challenges include: going to bed at inconsistent times (ranging from 10pm to 1am), using my phone in bed, and often feeling tired in the morning despite getting sleep. Please research the most effective, scientifically-backed sleep hygiene practices and in


Node: research_supervisor

Node: final_report_generation

FINAL REPORT GENERATED



# Comprehensive Sleep Improvement Plan: Evidence-Based Strategies for Better Sleep Quality

## Understanding Your Sleep Challenges

The sleep challenges you're experiencing—inconsistent bedtimes, phone use in bed, and morning fatigue—are interconnected issues that significantly impact sleep quality. Research from BMC Public Health found that 73.1% of college students have irregular bedtime schedules, with those experiencing high frequency irregular bedtimes showing significantly worse sleep quality scores and reduced average daily sleep time [1]. Your variable bedtime pattern (10pm-1am) places you in a category that research shows is associated with circadian rhythm disruption and poorer overall sleep outcomes.

A comprehensive study published in Sleep found that sleep irregularity operates differently than sleep duration in affecting well-being. While sleep duration predicted daily variations in mood and energy, sleep irregularity was the primary predictor of poor average well-being over time. For every additional hour of irregular sleep timing, participants reported significantly worse ratings on measures of happiness, health, and stress levels [2]. This explains why you may feel tired in the morning despite getting adequate sleep hours—the inconsistency itself is disrupting your sleep architecture.

## The Science Behind Screen Time and Sleep Disruption

Your habit of using your phone in bed is supported by extensive research as a significant contributor to sleep problems. A controlled study published in Sleep Medicine found that blue-light exposure for just one hour before bedtime significantly decreased the ratio of deep sleep compared to incandescent light or blue light-blocking glasses [3]. The blue light from smartphones, encompassing wavelengths between 400-500 nm, suppresses melatonin production by affecting intrinsic photosensitive retinal ganglion cells in your eyes.

Recent research tracking college students found that just two hours of evening screen exposure caused an average 1.1-hour circadian phase delay. Even more concerning, students who combined irregular sleep schedules with late-night electronic device use showed significant circadian disruption by the end of an academic term [4]. A large-scale study of over 122,000 participants found that daily screen use before bed was associated with a 33% higher prevalence of poor sleep quality and participants going to bed 9-15 minutes later depending on their chronotype [5].

However, it's important to note that some recent research suggests the relationship may be more complex than previously thought. A 2024 study from Toronto Metropolitan University found that overall sleep health was similar between nightly screen users and non-users, with occasional users (a few nights per week) actually showing the worst sleep outcomes [6]. This suggests that consistency in habits, even with screens, may be better than irregular patterns.

## Immediate Actionable Strategies

### Screen Time Management
Based on the research evidence, implement these immediate changes to your bedtime screen routine:

**Complete Digital Sunset Protocol**: Remove all screens from your bedroom environment entirely. Research shows that having internet-connected devices in bedrooms increases the risk of sleep troubles by 27% [7]. This includes phones, tablets, laptops, and televisions.

**90-Minute Buffer Zone**: Establish a strict 90-minute period before your target bedtime where no screens are used. Studies show that even 30 minutes of smartphone exposure with frequencies of 0.50-3.99 Hz reduces brain EEG activity, leading to mental disturbances and reduced sleep quality [8].

**Blue Light Management for Transition Period**: If you must use devices in the evening during your adjustment period, research supports using blue light-blocking glasses, which have been shown to maintain normal deep sleep ratios equivalent to incandescent lighting [3].

### Sleep Schedule Stabilization
Research consistently shows that sleep regularity is more important than many realize for overall health outcomes:

**Fixed Anchor Points**: Choose a consistent wake time first, then work backward 7.5-8 hours to establish your bedtime. Studies demonstrate that maintaining consistent sleep-wake times is crucial for circadian rhythm stability [9].

**Gradual Adjustment Method**: If your current bedtime varies widely (10pm-1am), adjust gradually by 15-30 minutes earlier each night rather than making dramatic changes. Research on circadian rhythm adjustment shows gradual shifts are more sustainable and less disruptive.

**Weekend Consistency**: Avoid "catch-up sleep" on weekends. NIH-supported research found that weekend recovery sleep does not effectively counteract the negative effects of irregular sleep patterns and can actually make sleep disturbances worse when returning to weekday schedules [10].

## Environmental Optimization Strategies

### Bedroom Environment
Create an environment that supports natural circadian rhythm regulation:

**Lighting Management**: Dim lights progressively in the 2-3 hours before bedtime to signal your brain to prepare for sleep. Research shows this gradual dimming supports natural melatonin production [11].

**Temperature Control**: Maintain a cool bedroom environment (65-68°F), as research indicates that blue light exposure affects not only melatonin but also core body temperature regulation, which is crucial for sleep onset [8].

**Technology-Free Sleep Sanctuary**: Studies of over 10,000 adolescents found that having any internet-connected device in the bedroom significantly increased sleep disturbances [7]. Apply this finding by charging devices outside your bedroom entirely.

### Pre-Sleep Routine Development
Based on sleep hygiene research, establish these evidence-based pre-sleep activities:

**Cognitive Transition Activities**: Replace screen time with activities that promote cognitive wind-down: reading physical books under warm lighting, journaling, gentle stretching, or meditation. Research shows these activities support the natural transition to sleep without the cognitive stimulation that screens provide [11].

**Consistent Routine Timing**: Implement the same sequence of activities each night, starting 60-90 minutes before your target bedtime. Studies on circadian rhythm regulation show that consistent behavioral cues help entrain your biological clock [9].

## Addressing Morning Fatigue

Your morning fatigue despite adequate sleep duration is likely related to sleep quality rather than quantity. Research provides several evidence-based explanations and solutions:

### Sleep Architecture Optimization
**Deep Sleep Protection**: Studies show that blue light exposure specifically reduces deep sleep ratios, which are crucial for feeling rested upon waking [3]. By eliminating evening screen time, you should see improvements in deep sleep quality within 1-2 weeks.

**Circadian Rhythm Alignment**: Research demonstrates that irregular sleep timing disrupts sleep architecture even when total sleep time is adequate [1]. Stabilizing your sleep schedule will improve the timing and quality of different sleep phases.

### Wake-Up Optimization
**Consistent Wake Times**: Even on weekends, maintain the same wake time. Studies show this is crucial for maintaining circadian rhythm stability and preventing the metabolic disruptions associated with irregular sleep patterns [10].

**Light Exposure Upon Waking**: Immediately expose yourself to bright light (preferably natural sunlight) upon waking. This helps maintain proper circadian rhythm entrainment and counteracts any residual circadian phase delays from previous irregular patterns [4].

## Long-Term Sleep Health Strategy

### Progressive Implementation Timeline

**Week 1-2: Foundation Building**
- Remove all screens from bedroom immediately
- Establish fixed wake time every day
- Implement 90-minute pre-bedtime screen cutoff

**Week 3-4: Schedule Stabilization**
- Gradually adjust bedtime to create consistent 7.5-8 hour sleep opportunity
- Develop and maintain consistent pre-sleep routine
- Monitor sleep quality improvements

**Week 5-8: Habit Reinforcement**
- Fine-tune environmental factors (temperature, lighting, noise)
- Address any remaining sleep quality issues
- Establish sustainable long-term patterns

### Monitoring and Adjustment
Research suggests tracking both sleep timing consistency and sleep quality outcomes. Studies show that apps can effectively monitor circadian patterns and sleep metrics comparable to clinical tools [12]. Track:

- Bedtime and wake time consistency
- Sleep onset time (how long to fall asleep)
- Night wakings frequency
- Morning energy levels and mood
- Daytime alertness patterns

### Addressing Potential Obstacles

**Social and Work Demands**: Research on college students and working adults shows that irregular schedules often stem from external demands. Plan strategies for maintaining sleep consistency even with variable social or work schedules, such as setting firm personal boundaries around bedtime regardless of external pressures.

**Gradual vs. Immediate Change**: While research supports immediate removal of screens from bedrooms, some individuals may need gradual adjustment. The key is consistency in whatever approach you choose—studies show that irregular implementation is worse than consistent moderate measures [6].

**Stress and Anxiety Management**: Research on nursing students found that perceived stress significantly impacts the relationship between circadian rhythms and sleep quality [13]. If anxiety about sleep changes or other stressors interfere with implementation, consider stress management techniques as part of your sleep improvement plan.

## Expected Timeline for Improvements

Based on the research evidence, you can expect to see improvements on the following timeline:

**1-3 days**: Reduced sleep onset time after removing screens from bedroom
**1-2 weeks**: Improved deep sleep ratios and reduced night wakings
**2-4 weeks**: Stabilized circadian rhythm and improved morning energy
**4-8 weeks**: Full optimization of sleep architecture and daytime functioning

Research consistently shows that the combination of addressing screen time, establishing consistent timing, and optimizing the sleep environment creates synergistic effects that are greater than implementing any single intervention alone.

### Sources

[1] Effects of an irregular bedtime schedule on sleep quality, daytime sleepiness, and fatigue among university students in Taiwan: https://bmcpublichealth.biomedcentral.com/articles/10.1186/1471-2458-8-301

[2] Irregular sleep and event schedules are associated with poorer self-reported well-being in US college students: https://academic.oup.com/sleep/article/42/12/zsz202/5549968

[3] Effects of pre-bedtime blue-light exposure on ratio of deep sleep in healthy young men: https://www.sciencedirect.com/science/article/abs/pii/S1389945721003257

[4] Blue Light Exposure From Electronic Devices on Sleep Disruption Among High School and College Student Populations: A Literature Review: https://www.chronobiologyinmedicine.org/journal/view.php?number=167&viewtype=pubreader

[5] Electronic Screen Use and Sleep Duration and Timing in Adults: https://pmc.ncbi.nlm.nih.gov/articles/PMC11950897/

[6] Using Your Phone at Night May Not Be as Bad as You Think, New Study Suggests: https://time.com/7335087/doom-scroll-phone-night-melatonin/

[7] Bedtime Screen Use Behaviors and Sleep Outcomes: Findings From the Adolescent Brain Cognitive Development (ABCD) Study: https://www.sciencedirect.com/science/article/pii/S2352721823000347

[8] Impact of Smartphone Usage on Sleep in Adolescents: A Clinically Oriented Review: https://www.cureus.com/articles/313319-impact-of-smartphone-usage-on-sleep-in-adolescents-a-clinically-oriented-review

[9] The importance of sleep regularity: a consensus statement of the American Academy of Sleep Medicine and Sleep Research Society: https://www.sleephealthjournal.org/article/S2352-7218%2823%2900166-3/fulltext

[10] Irregular sleep schedules can lead to bigger health issues: https://magazine.medlineplus.gov/article/irregular-sleep-schedules-can-lead-to-bigger-health-issues

[11] Screen time before bed: How tech affects sleep habits: https://www.missionhealth.org/healthy-living/blog/screen-time-before-bed-how-tech-affects-sleep-habits

[12] Examining Human-Smartphone Interaction as a Proxy for Circadian Rhythm in Patients With Insomnia: Observational Study: https://www.jmir.org/2023/1/e48044/

[13] Association Between Circadian Rhythm and Sleep Quality: A Cross-Sectional Study Among Nursing Interns: https://www.frontiersin.org/journals/neuroscience/articles/10.3389/fnins.2022.995775/full


Research workflow completed!


## Task 9: Understanding the Output

Let's break down what happened:

### Phase 1: Clarification
The system checked if your request was clear. Since you provided specific details about your sleep issues, it likely proceeded without asking clarifying questions.

### Phase 2: Research Brief
Your request was transformed into a detailed research brief that guides the supervisor's delegation strategy.

### Phase 3: Supervisor Delegation
The supervisor analyzed the brief and decided how to break down the research:
- Used `think_tool` to plan strategy
- Called `ConductResearch` to delegate to researchers
- Each delegation specified a focused research topic (e.g., sleep hygiene, circadian rhythm, blue light effects)

### Phase 4: Parallel Research
Researchers worked on their assigned topics:
- Each researcher used web search tools to gather information
- Used `think_tool` to reflect after each search
- Decided when they had enough information
- Compressed their findings into clean summaries

### Phase 5: Final Report
All research findings were synthesized into a comprehensive sleep improvement plan with:
- Well-structured sections
- Evidence-based recommendations
- Practical action items
- Sources for further reading

## Task 10: Key Takeaways & Next Steps

### Architecture Benefits
1. **Dynamic Decomposition** - Research structure emerges from the question, not predefined
2. **Parallel Efficiency** - Multiple researchers work simultaneously
3. **ReAct Quality** - Strategic reflection improves search decisions
4. **Scalability** - Handles token limits gracefully through compression
5. **Flexibility** - Easy to add new tools and capabilities

### When to Use This Pattern
- **Complex research questions** that need multi-angle investigation
- **Comparison tasks** where parallel research on different topics is beneficial
- **Open-ended exploration** where structure should emerge dynamically
- **Time-sensitive research** where parallel execution speeds up results

### When to Use Section-Based Instead
- **Highly structured reports** with predefined format requirements
- **Template-based content** where sections are always the same
- **Sequential dependencies** where later sections depend on earlier ones
- **Budget constraints** where token efficiency is critical

### Extend the System
1. **Add MCP Tools** - Integrate specialized tools for your domain
2. **Custom Prompts** - Modify prompts for specific research types
3. **Different Models** - Try different Claude versions or mix models
4. **Persistence** - Use a real database for checkpointing instead of memory

### Learn More
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Open Deep Research Repo](https://github.com/langchain-ai/open_deep_research)
- [Anthropic Claude Documentation](https://docs.anthropic.com/)
- [Tavily Search API](https://tavily.com/)

## ❓ Question #3:

What are the trade-offs of using parallel researchers vs. sequential research? When might you choose one approach over the other?

##### Answer:
Using parallel researchers means having multiple agents work on different parts of the problem at the same time. This is useful when we want results quickly or when the topic is broad and exploratory, because it gives us a wider range of ideas and sources. The downside is that it costs more resources and we need extra effort to combine and summarize the results, and sometimes agents may repeat the same work.
Sequential research works step by step, where each stage depends on the previous one. This approach is slower, but it is more focused and efficient, and it works well when the problem is clearly defined or when we need deeper reasoning. The risk is that if an early step is wrong, it can affect everything that comes after it. In many cases, combining both approaches works best, by starting with parallel research and then moving to a more structured, sequential process.

## ❓ Question #4:

How would you adapt this deep research architecture for a production wellness application? What additional components would you need?

##### Answer:
To use a deep research architecture in a real wellness app, the system would need more than just research agents. It would need to understand the user’s personal context and goals in order to give relevant and personalized advice, strong safety measures would also be important, especially because wellness apps deal with sensitive topics and should not give harmful or misleading guidance.
The system would also need to rely on trustworthy, evidence-based sources instead of assumptions or trends. Keeping long-term memory about the user’s progress would help the app track changes over time and adjust recommendations. Finally, feedback mechanisms, privacy protections, and ongoing monitoring would be necessary to make sure the system stays helpful, safe, and reliable in a production setting. A lot of the things we've seen in the previous assignments.

## 🏗️ Activity #2: Custom Wellness Research

Using what you've learned, run a custom wellness research task.

**Requirements:**
1. Create a wellness-related research question (exercise, nutrition, stress, etc.)
2. Modify the configuration for your use case
3. Run the research and analyze the output
4. Document what worked well and what could be improved

**Experiment ideas:**
- Research exercise routines for specific conditions (bad knee, lower back pain)
- Compare different stress management techniques
- Investigate nutrition strategies for specific goals
- Explore meditation and mindfulness research

**YOUR CODE HERE**

First of all, I had to switch to OpenAI models, because these were too expensive and I kept getting an error. 

**1. I tried with gpt-4.1-mini and gpt-4o-mini, with "max_concurrent_research_units": 1**

**What worked well**: The research request was clearly defined and focused on a realistic, common problem, daily stress related to work or studies, while also specifying concrete symptoms such as difficulty relaxing and intrusive thoughts at night. This clarity helped guide the research toward relevant, evidence-based stress management techniques rather than generic wellness advice.
The output successfully compared multiple well-established approaches, including mindfulness, cognitive-behavioral strategies, physical exercise, and relaxation techniques. Each technique was discussed in terms of benefits and limitations, which made the comparison balanced and informative. The inclusion of both psychological and physiological approaches provided a well-rounded perspective on stress management.
Another strength was the strong emphasis on evidence-based sources. The report consistently referenced meta-analyses, randomized controlled trials, and reputable health organizations, which increased the credibility and academic quality of the findings. The structured format, with clear sections and a comparative summary table, made the information easy to follow and supported practical decision-making.
Finally, the recommendations were tailored to specific needs, such as daily stress, evening relaxation, and sleep-related difficulties. This made the results more actionable and aligned well with the original wellness request.

**What could be improved**: The main drawback is that the report is quite long, which can make it harder to quickly identify the most important takeaways. Adding a short summary or a “top recommendations” section would make it more user-friendly.
The advice could also be more personalized. While the recommendations are generally helpful, they don’t fully take into account practical constraints like limited time, motivation, or personal preferences. A simple, example stress-management plan would make the results easier to apply in everyday life.
Finally, some techniques overlap in their effects, and the comparison could be clearer by ranking or prioritizing them based on effort, effectiveness, or how quickly they work. This would help readers decide which strategies to try first instead of feeling overwhelmed by too many options.

**2. Same models, but with 2 parallel researchers -> "max_concurrent_research_units": 2**

**What worked well**: Compared to the previous configuration, this setup produced a much more detailed and wide-ranging report. Allowing two concurrent research units resulted in a broader exploration of stress management techniques, including more emphasis on intrusive thoughts, sleep-related stress, and techniques like journaling and guided imagery that were less prominent before.
The explanations of each technique were more structured and consistent, with clear sections for overview, benefits, and limitations. This made the comparison between techniques easier and more systematic than in the earlier result. The discussion of CBT was also deeper and more convincing, especially in how it was linked to managing intrusive thoughts and improving sleep, which aligned well with the original wellness request. Overall, this version felt more “complete” and academically thorough than the previous output, making it better suited for a comprehensive comparison rather than a quick overview.

**What could be improved**: While this result is more comprehensive than the previous one, it is also noticeably longer and more repetitive. Some ideas, especially around mindfulness, intrusive thoughts, and CBT, overlap across sections, which likely comes from running parallel research units. This configuration is more resource-intensive than the previous one, and the added detail may not always justify the extra cost. For most practical purposes, the earlier setup may be sufficient, while this version is better suited when a deeper, more exhaustive comparison is explicitly required.

**3. I used gpt-5.2 for the "thinking" models, and I left 2 parallel researchers.**

**What worked well**: Compared to the first two configurations, this setup clearly produced the most targeted and actionable output. While the earlier results focused more on broadly explaining and comparing techniques, this version was much more decisive about what works best for this specific problem, like daily stress, difficulty unwinding, and nighttime rumination.
The biggest improvement is how strongly the recommendations are prioritized. Unlike the previous outputs, which presented techniques more evenly, this version clearly highlights CBT-I, mindfulness-based interventions, and relaxation skills as the most relevant options, based on both evidence strength and symptom fit. This makes the result feel more practical and easier to apply.
The reasoning quality is also noticeably higher. Compared to the earlier configs, this output does a better job explaining why certain techniques are better fits, not just that they work. The comparisons are more nuanced, especially when discussing trade-offs such as speed of benefit, adherence difficulty, and when professional support is needed.
Overall, this configuration feels less like a general literature review and more like a well-reasoned recommendation tailored to the user’s situation.

**What could be improved**: The main downside compared to the earlier configurations is that this output is significantly longer and more complex. While the depth and detail are impressive, it may be excessive if the goal is simply to compare techniques at a high level. In contrast, the first configuration was easier to skim, and the second provided a good middle ground between depth and readability.
Because the model is more “intelligent,” it also introduces more nuance, caveats, and conditional recommendations. While this improves accuracy, it can make decision-making harder for someone who just wants a short answer on where to start. A brief one-page summary or “recommended path” at the very top would improve usability.
Finally, this setup is the most resource-intensive of the three. Compared to the previous configs, the additional depth may not always justify the higher cost and runtime, especially for repeated or exploratory runs. This configuration is best suited for final, high-quality analysis rather than quick iteration.

**Overall Comparison**

Config 1 worked best for a clear, concise overview.

Config 2 offered broader coverage with more detail, but some redundancy.

Config 3 delivered the strongest reasoning and most actionable recommendations, at the cost of length and resources.

In [37]:
# YOUR CODE HERE
# Create your own wellness research request and run it

my_wellness_request = """
I want to improve how I manage stress in my daily life. I currently:
- Feel stressed almost every day, especially due to work or studies
- Have trouble relaxing after a long day
- Thoughts keep me awake at night

Please research and compare different evidence-based stress management techniques, explain their benefits and limitations.
"""

# Optionally modify the config
my_config = {
    "configurable": {
        "research_model": "openai:gpt-4.1-mini",
        "research_model_max_tokens": 10000,
        "compression_model": "openai:gpt-4o-mini",
        "compression_model_max_tokens": 8192,
        "final_report_model": "openai:gpt-4.1-mini",
        "final_report_model_max_tokens": 10000,
        "summarization_model": "openai:gpt-4o-mini",
        "summarization_model_max_tokens": 8192,
        "allow_clarification": True,
        "max_concurrent_research_units": 1,
        "max_researcher_iterations": 2,
        "max_react_tool_calls": 3,
        "search_api": "tavily",
        "max_content_length": 50000,
        "thread_id": str(uuid.uuid4())
    }
}
async def run_custom_research(request, config):
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user" and node_output:
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run your research
await run_custom_research(my_wellness_request, my_config)

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with researching evidence-based stress management techniques suitable for daily life stress related to work and studies, difficulties relaxing, and sleep disturbances due to stress-related thoughts. I will now begin the research process and provide a comparison of the techniques, including their benefits and limitations.

Node: write_research_brief

Research Brief Generated:
I want to explore and compare various evidence-based stress management techniques that are effective for daily life stress primarily caused by work or studies. Specifically, I experience stress almost every day, have difficulty relaxing after a long day, and have stress-related thoughts that interfere with my ability to sleep. Please include an evaluation of each technique’s benefits and limitations, considering how well they address daily stress, promote relaxation in the evening, and improve s...

Node: research_supe

# Comprehensive Review of Evidence-Based Stress Management Techniques for Daily Work and Study-Related Stress

This report explores various scientifically supported stress management techniques effective for managing daily stress primarily caused by work or studies. It evaluates each approach in terms of reducing daily stress, promoting evening relaxation, and improving sleep quality—particularly addressing common difficulties such as trouble relaxing after a long day and stress-related intrusive thoughts that interfere with sleep. The techniques discussed include mindfulness, cognitive-behavioral strategies, physical exercise, relaxation exercises, and other evidence-based methods. The findings are based on meta-analyses, randomized controlled trials, and guidelines from reputable health organizations.

---

## Understanding Work and Study-Related Stress

Chronic stress from work or academic demands is widespread and linked to adverse mental and physical health outcomes including anxiety, depression, decreased productivity, and poor sleep quality. Stress impairs relaxation after work hours and exacerbates negative, ruminative thoughts at bedtime, directly interfering with the ability to fall and stay asleep. Effective stress management is crucial to interrupt this cycle and improve overall well-being.

---

## Mindfulness-Based Interventions

### Overview
Mindfulness-Based Stress Reduction (MBSR) and related mindfulness practices involve fostering present-moment awareness through techniques such as breathing meditation, body scanning, and mindful movement. These cultivate nonjudgmental acceptance of thoughts and feelings, which can reduce the impact of stress-provoking cognitions.

### Benefits
- **Stress Reduction:** Numerous studies and meta-analyses show mindfulness reduces perceived stress levels and physiological stress markers such as cortisol among healthy adults [25][26].
- **Evening Relaxation:** Mindfulness practices help disengage from stress-related ruminations by training attention control, allowing relaxation in the evening hours [7][9].
- **Sleep Quality Improvement:** Mindfulness reduces pre-sleep cognitive arousal (worrying and racing thoughts), which improves subjective sleep quality and sleep satisfaction. Though objective sleep measures (e.g., polysomnography) show mixed results, improvements in sleep quality ratings and reductions in insomnia symptoms are documented [6][8][9].
- Mindfulness-based programs have strong evidence supporting their use for students and workers experiencing anxiety, depression, and sleep difficulties [4][9].

### Limitations
- Mindfulness requires consistent practice over weeks to months to achieve significant benefits, which may limit immediate relief.
- Effectiveness can vary; some individuals may find the focus on internal experience challenging or perceive mindfulness as less practical for acute stress.
- Not all objective sleep parameters improve, pointing to possible need for adjunctive treatments for severe insomnia [8].

---

## Cognitive-Behavioral Strategies (CBT)

### Overview
CBT targets maladaptive thought patterns and stress responses through cognitive restructuring and behavioral changes. It includes techniques such as identifying and challenging negative thoughts, problem-solving skills, relaxation training, and sleep hygiene education.

### Benefits
- **Daily Stress:** CBT is effective for work-related stress and anxiety by changing cognitive appraisals of stressors and fostering proactive coping behaviors [11][12][13].
- **Evening Relaxation:** CBT techniques include relaxation exercises and cognitive techniques such as stimulus control and worry time scheduling that can help reduce evening hyperarousal [14].
- **Sleep Quality:** CBT for insomnia (CBT-I), a specific form of CBT, has strong evidence to reduce sleep latency, decrease nighttime awakenings, and improve overall sleep quality and efficiency by targeting stress-related thoughts and habits that interfere with sleep [12][29].
- Adaptable for self-help or guided therapy, making it widely accessible.

### Limitations
- Requires learning and practice, often over several weeks.
- Access to trained therapists may be limited for some.
- Emotional effort and motivation are needed to restructure long-standing thought patterns.
- May be less effective if stressors are predominantly external and uncontrollable (e.g., high workload without feasible change).

---

## Physical Exercise

### Overview
Regular physical activity, including aerobic and resistance exercise, is widely recommended for stress management and sleep enhancement.

### Benefits
- **Stress Reduction:** Exercise promotes endorphin release and modulation of stress hormones (like cortisol), reducing perceived stress and anxiety [16][17].
- **Evening Relaxation:** Moderate exercise sessions promote relaxation in the evening if completed at least 3–4 hours before bedtime. Exercising helps burn off stress-related tension accumulated during the day [21][22].
- **Sleep Quality:** Consistent exercise improves sleep onset latency, increases slow-wave (deep) sleep, and enhances subjective sleep quality [19][20].
- Guidelines from health organizations emphasize physical activity as a cornerstone in managing stress and supporting mental health [34][35].

### Limitations
- Vigorous exercise too close to bedtime may disrupt sleep onset and architecture.
- Some individuals may find motivation or time constraints challenging.
- Exercise effects on stress can be acute and cumulative; inconsistent routines may limit benefits.
- Overtraining without adequate rest can increase stress.

---

## Relaxation Techniques

### Overview
These include progressive muscle relaxation (PMR), deep breathing exercises, guided imagery, and autogenic training.

### Benefits
- **Stress Reduction:** Relaxation techniques decrease sympathetic nervous system arousal, lowering heart rate, blood pressure, and cortisol levels [25].
- **Evening Relaxation:** Directly target physiological tension, facilitating relaxation and mental calmness after a stressful day [5].
- **Sleep Improvement:** Relaxation before bedtime helps reduce cognitive and somatic arousal, improving subjective sleep quality, especially for stress-related insomnia [7][8].
- Simple and can be easily integrated into nightly routines or short breaks during the day.

### Limitations
- May have less impact on the underlying cognitive causes of stress (compared to CBT).
- Immediate benefits may be short-lived without consistent practice.
- Requires learning proper technique to maximize benefit.

---

## Other Evidence-Based Approaches

### Social Support and Positive Lifestyle Changes
- Engaging in social activities and fostering supportive relationships buffer stress impacts and improve mood [3].
- Nutritional considerations, adequate hydration, limiting caffeine or alcohol, and maintaining a routine contribute to stress resilience and sleep quality [2][17].

### Yoga and Tai Chi
- Combines mindfulness, physical activity, and relaxation.
- Evidence supports improvements in stress, anxiety, and sleep, particularly among individuals with chronic conditions or elevated stress levels [2].

### Hypnosis and Meditation
- Techniques such as guided hypnosis have shown promise in reducing stress and improving relaxation, though more high-quality studies are needed.

### Workplace and Study Environment Interventions
- Organizational strategies such as workload management, flexible schedules, and stress education benefit mental health but are outside individual behavioral management [1][14].

---

## Comparative Summary Table

| Technique                     | Daily Stress Reduction                | Evening Relaxation                 | Sleep Quality Improvement        | Limitations                        |
|-------------------------------|-------------------------------------|----------------------------------|---------------------------------|----------------------------------|
| **Mindfulness (MBSR)**         | High – reduces rumination and stress perception | Moderate – aids mental calmness   | Moderate – improves subjective sleep, less on objective | Requires time/practice, variable effect |
| **Cognitive-Behavioral Therapy (CBT & CBT-I)** | High – changes cognitive distortions directly | High – teaches relaxation & cognitive control | High – strongest evidence for insomnia treatment | Needs time, skills, possible therapist access |
| **Physical Exercise**           | High – modulates hormones and mood  | Moderate – helps if timed properly | High – improves sleep latency and depth | Evening timing critical, motivation needed |
| **Relaxation Exercises**        | Moderate – reduces physiological arousal | High – directly induces relaxation | Moderate – reduces arousal and sleep latency | Less effect on cognitive stress, requires practice |
| **Other (Yoga/Tai Chi, Social Support)** | Moderate – combines multiple mechanisms | Moderate – physical + mental benefits | Moderate – some evidence of sleep improvement | Varies by individual, less standardized |

---

## Recommendations Based on Specific Needs

- **For persistent daily stress:** Combining mindfulness with CBT offers synergistic cognitive and emotional regulation benefits.
- **For improving evening relaxation:** Relaxation exercises such as deep breathing or progressive muscle relaxation can be practiced immediately after work or study.
- **For sleep difficulties caused by stress-related thoughts:** CBT for insomnia (CBT-I) and mindfulness meditation are most effective in reducing cognitive arousal before sleep.
- **To enhance overall resilience:** Incorporate regular physical exercise, preferably earlier in the day, alongside positive lifestyle habits and healthy social support.

---

## Conclusion

Effective management of daily life stress related to work or studies demands a multidimensional approach integrating evidence-based methods. Mindfulness and CBT have the strongest evidence for not only reducing stress but also improving relaxation and sleep quality, particularly by addressing cognitive-emotional factors. Physical exercise and relaxation exercises complement cognitive approaches by targeting physiological arousal and mood. Personal preferences, availability, and consistency will influence which techniques work best, but integrating several methods typically yields the most comprehensive benefits.

---

### Sources

[1] Evidence-Based Strategies to Reduce Workplace Stress: https://www.knowyourmindconsulting.com/post/stress-management-for-employees  
[2] 15 Evidence-Based Ways to Relieve Stress: https://performancehealthcenter.com/2022/05/15-evidence-based-ways-to-relieve-stress/  
[3] Stress management: Ways to cope with stressors large and small: https://med.stanford.edu/news/insights/2025/12/stress-management-coping-skills-and-tools.html  
[4] Stress Management (MBSR overview): https://www.ncbi.nlm.nih.gov/books/NBK513300/  
[5] Stress Management: Strategies to Deal with Stress: https://www.helpguide.org/mental-health/stress/stress-management  
[6] Mindfulness Improves Sleep and Reduces Stress: https://neurosciencenews.com/mindfulness-sleep-stress-26364/  
[7] The role of mindfulness and relaxation in improved sleep quality: https://pmc.ncbi.nlm.nih.gov/articles/PMC8653783/  
[8] Impact of Mindfulness-Based Stress Reduction on sleep-related outcomes: https://www.sciencedirect.com/science/article/pii/S2212958824000879  
[9] Effect of a mindfulness program on stress, anxiety, depression, sleep: https://www.frontiersin.org/journals/psychology/articles/10.3389/fpsyg.2025.1508934/full  
[10] Mindfulness exercises - Mayo Clinic: https://www.mayoclinic.org/healthy-lifestyle/consumer-health/in-depth/mindfulness-exercises/art-20046356  
[11] CBT for Workplace Wellbeing, Work Related Stress and Anxiety: https://www.maudsleylearning.com/insights/blogs-and-videos/cbt-for-workplace-wellbeing-work-related-stress-and-anxiety/  
[12] 10 Simple and Effective CBT Life Hacks to Manage Workplace Stress: https://lightonanxiety.com/cbt-treatment/10-simple-and-effective-cbt-life-hacks-to-manage-workplace-stress/  
[13] CBT Tips for Coping With Workplace Stress: https://bayareacbtcenter.com/how-cbt-can-help-you-better-handle-stress-within-the-workplace/  
[14] Effective Strategies for Managing Work-Related Stress: https://therapygroupdc.com/therapist-dc-blog/effective-strategies-for-managing-work-related-stress/  
[16] Exercise as a Stress Buster | How Physical Activity Reduces Stress: https://www.norcen.org/services/mental-health/stress-awareness-month/exercise-as-a-stress-buster/  
[17] Nutrition, Exercise, and Sleep Tips for Stress Relief - Redeemer Health: https://www.redeemerhealth.org/stories/health-benefits-stress-reduction-nutrition-exercise-and-sleep-tips-stress-relief  
[19] The Effect of Physical Activity on Sleep Quality and Sleep Disorder: https://pmc.ncbi.nlm.nih.gov/articles/PMC10503965/  
[20] The impact of exercise on sleep and sleep disorders - Nature: https://www.nature.com/articles/s44323-024-00018-w  
[21] The Impact of Evening Exercise on Sleep: https://observatoireprevention.org/en/2025/08/06/the-impact-of-evening-exercise-on-sleep/  
[22] Dose-response relationship between evening exercise and sleep: https://pmc.ncbi.nlm.nih.gov/articles/PMC12000559/  
[25] Effectiveness of stress management interventions to change cortisol levels: https://www.sciencedirect.com/science/article/pii/S0306453023003931  
[26] Stress management techniques: Are they all equivalent, or do they have specific effects: https://www.researchgate.net/publication/15318139_Stress_management_techniques_Are_they_all_equivalent_or_do_they_have_specific_effects  
[29] A systematic review and meta-analysis of psychological interventions: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0317125  
[34] Executive summary - WHO guidelines on mental health at work: https://www.ncbi.nlm.nih.gov/books/n/who363177/fm-ch1/  
[35] STRESS...At Work (99-101) | NIOSH - CDC: https://www.cdc.gov/niosh/docs/99-101/default.html


Research workflow completed!


In [38]:
my_config1 = {
    "configurable": {
        "research_model": "openai:gpt-4.1-mini",
        "research_model_max_tokens": 10000,
        "compression_model": "openai:gpt-4o-mini",
        "compression_model_max_tokens": 8192,
        "final_report_model": "openai:gpt-4.1-mini",
        "final_report_model_max_tokens": 10000,
        "summarization_model": "openai:gpt-4o-mini",
        "summarization_model_max_tokens": 8192,
        "allow_clarification": True,
        "max_concurrent_research_units": 2,
        "max_researcher_iterations": 2,
        "max_react_tool_calls": 3,
        "search_api": "tavily",
        "max_content_length": 50000,
        "thread_id": str(uuid.uuid4())
    }
}
await run_custom_research(my_wellness_request, my_config1)

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with your request. You want me to research and compare various evidence-based stress management techniques, focusing on their benefits and limitations, particularly related to daily stress, difficulty relaxing, and sleep disturbances caused by stress. I will now begin the research process.

Node: write_research_brief

Research Brief Generated:
I want to research and compare evidence-based stress management techniques that can help me manage my nearly daily stress, particularly stress caused by work or studies. The research should focus on techniques that improve my ability to relax after a long day and reduce intrusive thoughts that keep me awake at night. I want a detailed comparison of each technique's benefits and limitations, considering their effectiveness for daily stress relief, relaxation facilitation, and improving sleep qual...

Node: research_supervisor

Node: final_report_gener

# Evidence-Based Stress Management Techniques for Daily Work- or Study-Related Stress: A Comprehensive Comparison

Managing nearly daily stress caused by work or studies requires effective strategies that not only relieve stress but also facilitate relaxation after a long day and help reduce intrusive thoughts that interfere with sleep. This report presents a detailed, evidence-based analysis of different stress management techniques widely supported by scientific research. Each technique’s benefits and limitations are discussed with respect to daily stress relief, relaxation facilitation, and sleep quality improvement.

---

## 1. Mindfulness Meditation

### Overview
Mindfulness meditation involves focusing attention on the present moment, often through controlled breathing and non-judgmental awareness of thoughts and sensations.

### Benefits
- **Stress Reduction:** Regular mindfulness meditation has been shown to significantly lower perceived stress and anxiety levels. Studies report improvements in cognitive flexibility and emotional regulation after just a few weeks of practice ([1](https://www.nature.com/articles/s41598-025-23727-z)).
- **Relaxation Facilitation:** Mindfulness promotes calmness by improving attentional control and reducing rumination, which helps the mind transition more smoothly into relaxation post-work or study.
- **Sleep Improvement:** Mindfulness decreases intrusive and repetitive thoughts that often plague the mind at night, thereby enhancing sleep onset and quality ([13](https://mcpress.mayoclinic.org/mental-health/coping-with-unwanted-and-intrusive-thoughts/)).

### Limitations
- **Adherence Challenges:** Long-term commitment to mindfulness practice can be difficult due to motivation or time constraints.
- **Effectiveness Variability:** Some individuals may experience slower or less pronounced benefits, especially if initial stress levels are very high.

---

## 2. Progressive Muscle Relaxation (PMR)

### Overview
PMR is a technique involving sequential tensing and relaxing of major muscle groups to create deep physical relaxation.

### Benefits
- **Effective Stress Relief:** PMR alone significantly reduces physiological arousal related to stress, as evidenced by changes in electrodermal activity and reduced anxiety ([4](https://pmc.ncbi.nlm.nih.gov/articles/PMC8272667/)).
- **Improved Sleep Quality:** When combined with mindfulness meditation, PMR shows enhanced improvements in sleep disorders and reductions in anxiety and depressive symptoms ([2](https://www.frontiersin.org/journals/psychiatry/articles/10.3389/fpsyt.2025.1542028/full)).
- **Easily Learned and Applied:** It can be performed almost anywhere without special equipment.

### Limitations
- **Time Requirement:** Each session can take 15-30 minutes, which may deter regular practice for busy individuals.
- **Initial Guidance Needed:** Proper instruction is necessary to avoid incomplete muscle relaxation or tensing.

---

## 3. Deep Breathing Exercises

### Overview
These exercises involve slow, controlled breathing patterns to stimulate the parasympathetic nervous system, encouraging relaxation.

### Benefits
- **Rapid Stress Reduction:** Deep breathing helps quickly activate the body’s relaxation response, lowering heart rate and emotional stress ([2](https://www.fitonhealth.com/blog/the-14-best-tips-for-stress-management-backed-by-science)).
- **Improves Emotional Balance:** Regular practice can reduce emotional reactivity and help regulate intrusive thoughts.
- **Convenient and Versatile:** Can be applied anytime, such as immediately after work or before bedtime.

### Limitations
- **Short-term Effects:** Effects may be brief unless practiced regularly.
- **Technique Sensitivity:** Incorrect breathing patterns can sometimes cause lightheadedness or discomfort.

---

## 4. Yoga

### Overview
Yoga combines physical postures, breathing techniques, and meditation to promote overall well-being.

### Benefits
- **Comprehensive Stress Reduction:** Yoga reduces cortisol levels and anxiety symptoms, thereby lowering stress from work or studies ([3](https://www.ncbi.nlm.nih.gov/books/NBK513300/)).
- **Promotes Physical and Mental Relaxation:** The holistic approach enhances bodily relaxation and calms the mind, facilitating easier transition into restful states.
- **Enhances Sleep Quality:** Yoga practice improves sleep latency and duration by addressing both muscular tension and stress-related hyperarousal.

### Limitations
- **Skill and Time Investment:** Requires learning proper form and regular time commitment.
- **Physical Limitations:** Not suitable for everyone, especially those with physical injuries without modifications.

---

## 5. Cognitive Behavioral Therapy (CBT)

### Overview
CBT addresses stress by changing negative thought patterns and behaviors. It often integrates relaxation training and sleep hygiene strategies.

### Benefits
- **Highly Effective for Anxiety and Sleep:** Meta-analyses show CBT surpasses other relaxation techniques in reducing anxiety and improving sleep quality ([17](https://pubmed.ncbi.nlm.nih.gov/29037266/), [16](https://peerj.com/articles/17491.pdf)).
- **Reduces Intrusive Thoughts:** CBT includes cognitive restructuring and thought-stopping techniques, which actively reduce mental rumination and intrusive thoughts that interfere with sleep ([9](https://positivepsychology.com/thought-stopping-techniques/), [10](https://mcpress.mayoclinic.org/mental-health/coping-with-unwanted-and-intrusive-thoughts/)).
- **Long-Term Benefits:** Offers durable improvements by addressing root cognitive causes of stress.

### Limitations
- **Professional Guidance Required:** Typically involves working with a trained therapist, which may have cost or accessibility barriers.
- **Time and Effort Intensive:** Requires active participation and homework, which can be challenging for those with busy schedules.

---

## 6. Managing Intrusive Thoughts and Sleep Quality

### Overview
Intrusive thoughts are unwanted, distressing mental images or ideas that commonly increase with stress and poor sleep.

### Benefits of Managing Intrusive Thoughts
- **Improved Sleep:** Reducing intrusive thoughts helps improve slow-wave sleep and REM sleep, promoting cognitive restoration ([22](https://www.scientificamerican.com/article/bad-sleep-leaves-the-brain-more-vulnerable-to-intrusive-thoughts/)).
- **Better Stress Control:** Techniques such as thought-stopping and mindfulness reduce pre-sleep cognitive arousal, making it easier to fall asleep ([11](https://pmc.ncbi.nlm.nih.gov/articles/PMC9552939/), [13](https://mcpress.mayoclinic.org/mental-health/coping-with-unwanted-and-intrusive-thoughts/)).

### Limitations
- **Techniques Alone May Not Suffice:** Often need to be combined with broader stress management or therapy approaches for best results.
- **Variable Effects:** Some individuals may require more intensive intervention if intrusive thoughts are severe or linked to clinical conditions.

---

## 7. Additional Techniques: Guided Imagery and Journaling

### Overview
- **Guided Imagery** involves visualizing calming scenes or experiences to promote mental relaxation.
- **Journaling** helps by externalizing worries, thus decreasing mental load.

### Benefits
- Both approaches reduce psychological distress and facilitate relaxation, supporting stress relief after a long day ([6](https://medium.com/@bnascimento_en/6-evidence-based-techniques-for-stress-management-995cb8ccd141)).
- Journaling before bedtime can reduce intrusive thoughts by organizing and processing emotions.

### Limitations
- Limited direct evidence for sleep improvement compared to other techniques.
- Relies heavily on individual preference and engagement.

---

## 8. Comparison Summary

| Technique                   | Daily Stress Relief                   | Relaxation Facilitation           | Sleep Quality Improvement               | Limitations                                  |
|-----------------------------|-------------------------------------|----------------------------------|----------------------------------------|----------------------------------------------|
| Mindfulness Meditation       | Moderate to High                    | High                             | Moderate to High                       | Requires consistent practice, motivation     |
| Progressive Muscle Relaxation| Moderate                          | High                             | Moderate to High (best combined)       | Time intensive, needs guidance                |
| Deep Breathing Exercises     | Moderate                          | Moderate to High                 | Moderate                              | Short-lived effects, technique sensitivity    |
| Yoga                        | High                              | High                             | Moderate to High                      | Requires time and physical ability            |
| Cognitive Behavioral Therapy | High                             | Moderate (via relaxation training)| High                                | Professional guidance needed, time-intensive |
| Intrusive Thought Management | Indirect but essential             | Supports relaxation indirectly   | High (when combined with other methods)| Often requires multi-approach intervention   |
| Guided Imagery & Journaling   | Moderate                         | Moderate                        | Limited                              | Less evidence for sleep, varies by individual |

---

## Conclusion

For daily stress primarily related to work or studies, multiple evidence-based techniques offer distinct advantages:

- **Mindfulness meditation and progressive muscle relaxation** are powerful for reducing daily stress and facilitating relaxation, with significant secondary benefits for sleep.
- **Deep breathing and yoga** provide convenient methods to quickly induce calmness physically and mentally.
- **Cognitive Behavioral Therapy (CBT)** remains a superior option for addressing stress-induced anxiety and sleep disturbances by targeting cognitive patterns and teaching effective intrusive thought management.
- Combining techniques (e.g., mindfulness with PMR) appears to enhance benefits, especially for improving sleep quality.
- Managing intrusive thoughts is crucial to prevent stress from disrupting sleep, and methods such as thought-stopping, mindfulness, and CBT provide robust tools to do so.

Choosing the right approach depends on individual preferences, available time, physical capability, and access to professional support. Consistency and integration of these techniques into daily routine are key factors for sustained stress relief, enhanced relaxation, and better sleep outcomes.

---

## Sources

[1] Effects of mindfulness breathing meditation on stress and cognitive functions: https://www.nature.com/articles/s41598-025-23727-z  
[2] Effects of mindfulness meditation combined with progressive muscle relaxation: https://www.frontiersin.org/journals/psychiatry/articles/10.3389/fpsyt.2025.1542028/full  
[3] Effectiveness of Yoga for stress reduction: https://www.ncbi.nlm.nih.gov/books/NBK513300/  
[4] Effectiveness of Progressive Muscle Relaxation: https://pmc.ncbi.nlm.nih.gov/articles/PMC8272667/  
[6] 6 Evidence-Based Techniques for Stress Management: https://medium.com/@bnascimento_en/6-evidence-based-techniques-for-stress-management-995cb8ccd141  
[9] Thought-stopping Techniques: https://positivepsychology.com/thought-stopping-techniques/  
[10] How to Stop Intrusive Thoughts: https://mcpress.mayoclinic.org/mental-health/coping-with-unwanted-and-intrusive-thoughts/  
[11] Strategies for Controlling Unwanted Intrusive Thoughts: https://pmc.ncbi.nlm.nih.gov/articles/PMC9552939/  
[13] Mindfulness and Intrusive Thoughts: https://mcpress.mayoclinic.org/mental-health/coping-with-unwanted-and-intrusive-thoughts/  
[16] CBT for Sleep Improvement: https://peerj.com/articles/17491.pdf  
[17] Meta-analysis on CBT for Anxiety and Sleep: https://pubmed.ncbi.nlm.nih.gov/29037266/  
[22] Bad Sleep and Vulnerability to Intrusive Thoughts: https://www.scientificamerican.com/article/bad-sleep-leaves-the-brain-more-vulnerable-to-intrusive-thoughts/


Research workflow completed!


In [39]:
my_config2 = {
    "configurable": {
        "research_model": "openai:gpt-5.2",
        "research_model_max_tokens": 10000,
        "compression_model": "openai:gpt-4o-mini",
        "compression_model_max_tokens": 8192,
        "final_report_model": "openai:gpt-5.2",
        "final_report_model_max_tokens": 10000,
        "summarization_model": "openai:gpt-4o-mini",
        "summarization_model_max_tokens": 8192,
        "allow_clarification": True,
        "max_concurrent_research_units": 2,
        "max_researcher_iterations": 2,
        "max_react_tool_calls": 3,
        "search_api": "tavily",
        "max_content_length": 50000,
        "thread_id": str(uuid.uuid4())
    }
}
await run_custom_research(my_wellness_request, my_config2)

Starting research workflow...


Node: clarify_with_user

I have enough information to proceed. You’re looking for a research-based comparison of evidence-based stress management techniques that can help with near-daily work/study stress, difficulty relaxing after the day, and nighttime rumination/insomnia—along with clear benefits and limitations of each approach. I’ll now begin the research and synthesize the findings into a comparative report.

Node: write_research_brief

Research Brief Generated:
What evidence-based stress management techniques are most effective for helping me reduce near-daily work/study stress, relax after a long day, and address nighttime rumination that keeps me awake, and how do these techniques compare in terms of (1) quality/strength of evidence (e.g., RCTs/meta-analyses), (2) typical time-to-benefit and required practice time, (3) effect sizes or clinically meaningful outcomes for stress, anxiety, and sleep/insomnia measures, (4) suitability for self-guided


Node: research_supervisor

Node: final_report_generation

FINAL REPORT GENERATED



# Evidence-Based Techniques to Reduce Near-Daily Work/Study Stress, Unwind After the Day, and Stop Nighttime Rumination (with Comparisons)

## What tends to work best for your specific pattern (daily stress + can’t unwind + bedtime rumination/insomnia)

**Most directly supported “best fit” options (highest relevance + strongest evidence):**
1. **CBT for Insomnia (CBT‑I) / digital CBT‑I** for nighttime rumination that keeps you awake, difficulty falling asleep, and fragmented sleep. Major guidelines recommend CBT‑I as *first-line* for chronic insomnia, and RCTs/meta-analyses show clinically meaningful improvements in insomnia severity and sleep continuity. Key components (stimulus control, sleep restriction) have strong evidence; “sleep hygiene alone” is not effective as a stand‑alone treatment. Sources: ACP guideline and AASM behavioral treatment guideline [1–3].  
2. **Mindfulness-based interventions (MBSR and related workplace/digital mindfulness programs)** for near-daily work/study stress and rumination. Meta-analyses show small–moderate improvements in stress and anxiety for internet programs and moderate–large effects in some workplace MBIs; also meaningful reductions in ruminative thinking. Sources [4–7].  
3. **Relaxation skills (especially Progressive Muscle Relaxation + diaphragmatic breathing)** for fast “downshifting” after work and pre-sleep arousal reduction. Evidence is generally positive; effects are typically smaller than CBT‑I for clinical insomnia but can be very practical for daily unwinding and sleep onset. Sources [8–9].  
4. **Exercise (aerobic; plus mind–body movement like tai chi)** improves stress, mood, and sleep quality; effects often build over weeks. Exercise may be less “targeted” than CBT‑I for insomnia but is broadly beneficial and durable. Sources [10–12].  
5. **ACT-based approaches** (Acceptance & Commitment Therapy), including ACT tailored for insomnia (ACT‑I), appear comparable to CBT‑I in some studies and may be especially useful when rumination is driven by **struggling with thoughts** (e.g., “I must fall asleep now”). Evidence base is growing but smaller than CBT‑I. Sources [13–14].  

**Helpful adjuncts (often smaller effects, but low cost and easy to add):**
- **Expressive writing / journaling** (small benefits; can help offload worry) [15].  
- **Time-management / organizational interventions** (can reduce upstream workload stress; evidence suggests associations and some interventions help, but studies vary and effects depend on implementation) [16–17].  
- **HRV biofeedback** (promising moderate effects for stress-related outcomes; costs/gear can be a barrier; evidence quality varies) [18–19].

---

## Comparison table (evidence strength, time-to-benefit, outcomes, practicality, risks, adherence)

### Key: evidence strength
- **High** = multiple meta-analyses + clinical guidelines / many RCTs
- **Moderate** = meta-analyses with heterogeneity or fewer RCTs
- **Emerging** = fewer trials or mixed quality

| Option | Best for your goals | Evidence quality/strength | Typical time-to-benefit & practice dose | Outcomes / effect sizes reported in research | Self-guided vs therapist | Accessibility/cost | Risks/contraindications | Common limitations/adherence issues |
|---|---|---|---|---|---|---|---|---|
| **CBT‑I (in-person)** | Bedtime rumination/insomnia; sleep onset & maintenance; “can’t switch off” | **High** (guidelines + meta-analyses) [1–3] | Often **2–6 weeks** for clear change; standard courses ~**4–8 sessions**; daily sleep diary | Meta-analyses show meaningful improvements in insomnia severity, sleep efficiency, SOL/WASO (varies by study) [1–3] | Therapist improves tailoring | Higher cost; availability varies | Sleep restriction can increase daytime sleepiness initially; caution for bipolar/mania risk, seizure risk, safety-critical jobs—needs clinician oversight if complex | Hardest part is adhering to sleep restriction/stimulus control; short-term fatigue can reduce motivation |
| **Digital CBT‑I (dCBT‑I: Sleepio/SHUTi-like)** | Same as above; scalable | **High–Moderate** (strong RCT evidence; comparable efficacy to in-person in reviews) [20–22] | Often **weeks**; modules over **6–8 weeks**; brief daily tasks | RCTs/reviews show significant ISI reductions; reported SOL reductions (e.g., ~15 min) and small TST gains in some analyses [20] | Often fully self-guided; sometimes with coaching | Usually subscription; cheaper than therapy | Same as CBT‑I; also “algorithm mismatch” for unusual schedules | Dropout can be substantial; requires consistent logging |
| **Mindfulness (MBSR / workplace MBIs)** | Daily stress, rumination, emotional reactivity; unwinding | **Moderate–High** (many RCTs/meta-analyses; workplace evidence notable) [4–7] | **2–8 weeks**; classic MBSR is **8 weeks** + home practice (often 20–45 min/day); shorter formats exist | Workplace review: stress reduction **SMD ~0.72**; mental health **SMD ~0.67** in some analyses [5]. Rumination reduction **SMD ~‑0.53** across RCTs [6] | Self-guided apps possible; groups/teachers help adherence and skill | Apps low cost; courses moderate | Can temporarily increase awareness of distress; caution with severe trauma/PTSD without support | Biggest barrier is **practice time**; benefits depend on consistency |
| **Internet mindfulness (iMBIs)** | Stress + mild anxiety; some rumination | **Moderate** (meta-analysis; generally small–moderate effects) [4] | Often **2–8 weeks**; 10–20 min/day typical | Stress **g ~‑0.39**, anxiety **g ~‑0.20** (small) [4] | Usually self-guided | Low cost | Similar as above | Engagement often low; effects smaller than intensive formats |
| **Progressive Muscle Relaxation (PMR)** | Rapid after-work decompression; sleep onset calming | **Moderate** (controlled studies; some recent RCT-like work) [8–9] | Often immediate short-term relief; **10–30 min**, 4–7 days/week; can be “micro-PMR” 2–5 min | Studies show reduced anxiety and improved sleep quality in some populations [9] | Easily self-guided via audio | Free/low cost | Generally safe; caution if muscle injury/pain conditions (modify tensing) | People rush it or stop once they feel a bit better; needs repetition |
| **Diaphragmatic breathing / paced breathing** | Quick downshift; pre-sleep arousal | **Moderate** (broad literature; many protocols; effect sizes vary) [8] | Immediate calming; **2–10 min** sessions; multiple daily | Improves perceived relaxation; best as adjunct | Self-guided | Free | Hyperventilation/lightheadedness if done too forcefully; panic sufferers may need gentle approach | Under-dosing (too short/rare) or “trying to force relaxation” |
| **Guided imagery** | “Transition ritual” after work; pre-sleep | **Moderate** (varied studies) [8] | Often immediate; **5–20 min** | Improves relaxation states vs control in some studies [8] | Self-guided audio | Free/low cost | Some find imagery activating; trauma imagery caution | Fit/preference dependent |
| **ACT / ACT‑I (ACT for insomnia)** | Rumination driven by control/struggle; values-based stress coping | **Emerging–Moderate** (growing trials; fewer than CBT‑I) [13–14] | Often **4–8 weeks**; brief daily exercises (defusion, acceptance, values actions) | Pilot RCTs suggest improvements in insomnia severity and sleep-related QoL; sometimes similar to CBT‑I [13–14] | Can be self-guided; often better with therapist/coach | Books/apps moderate; therapy cost | If depression is severe or suicidality, professional support needed | People misunderstand acceptance as “resignation”; adherence to values work can drop under workload |
| **Exercise (aerobic; Pilates; tai chi)** | General stress resilience; mood; sleep quality | **Moderate** (many studies; heterogeneity; some strong trials) [10–12] | Stress effects can occur quickly after a session; sleep benefits typically **4–12+ weeks**; aim **150 min/week** moderate aerobic | Aerobic exercise meta-analytic sleep quality improvements around **SMD ~‑0.52** in one review [10]. Tai chi vs CBT‑I trial: tai chi may be inferior at 3 months but **non-inferior by 12 months** in one RCT [12] | Self-guided | Low–moderate cost | Injury risk; overtraining; late-evening high-intensity may worsen sleep for some | Consistency; scheduling; “all-or-nothing” mindset |
| **HRV biofeedback** | Physiological stress regulation; anxiety reduction | **Emerging–Moderate** (meta-analytic signals; quality varies) [18–19] | Often **2–8 weeks**; 10–20 min/day | Meta-analytic effect sizes reported around **g ~0.38** for some mental health outcomes; stress/anxiety improvements reported [18–19] | Often self-guided with device/app; clinician can optimize | Costs vary (device/app) | Some may feel anxiety focusing on body signals | Requires equipment + regular practice; technique quality matters |
| **Expressive writing / journaling** | Cognitive offloading; worry processing | **Moderate** (meta-analysis shows small effect) [15] | Often delayed benefit; **15–20 min**, 3–4 times/week for a few weeks | Meta-analysis: **small but significant** improvements for anxiety/stress/depression; benefits can be delayed [15] | Self-guided | Free | Can temporarily increase distress; avoid intense trauma writing alone if destabilizing | People stop when it feels uncomfortable; wrong timing (too close to bedtime can activate) |
| **Time management / organizational** | Upstream stress reduction; prevents evening catch-up | **Emerging–Moderate** (mixed; some reviews show improved well-being and stress associations) [16–17] | Benefits can be quick (days–weeks) once implemented | Reviews suggest planning/prioritizing relates to reduced stress and better well-being/performance (effect sizes vary; methods heterogeneous) [16–17] | Self-guided or coached | Low cost | None specific; can worsen perfectionism if rigid | Over-planning, unrealistic schedules; requires behavior change + boundary-setting |

---

## Deep dive by intervention (what it is, what it helps most, and what to watch for)

## CBT-based approaches (CBT for stress/anxiety; CBT‑I for sleep)

### CBT‑I (core for bedtime rumination + insomnia)
**What it targets:** the insomnia “maintenance loop” (conditioned arousal in bed, irregular sleep timing, worry about sleep, compensatory habits like long naps).  

**Guideline status:**  
- ACP recommends **CBT‑I as the initial treatment** for chronic insomnia disorder [1].  
- AASM guideline emphasizes CBT‑I/behavioral and psychological treatments; also highlights that **sleep hygiene should not be used alone** [2–3].

**Most evidence-supported components (and why they help rumination):**
- **Stimulus control**: re-associates bed with sleep rather than thinking/scrolling/worrying [2].  
- **Sleep restriction therapy**: consolidates sleep drive, reduces time awake in bed; network analyses identify it as among the most potent components [3].  
- **Cognitive techniques**: restructure catastrophic sleep thoughts and reduce pre-sleep worry.  
- **Wind-down + relapse prevention**: reduces evening arousal and helps long-term maintenance.

**Time-to-benefit:** often noticeable within **2–4 weeks**, with continuing improvements across a typical **6–8 week** protocol (digital or therapist-led).

**Who may want clinician oversight rather than purely self-guided:**  
- Complex sleep issues (suspected sleep apnea, parasomnias, severe circadian disorder, bipolar disorder/mania risk, seizure disorder, safety-critical work requiring high alertness), because sleep restriction can temporarily increase sleepiness and can be destabilizing in some conditions.

### Digital CBT‑I (dCBT‑I)
**Evidence:** Reviews and RCT evidence indicate dCBT‑I can significantly improve insomnia outcomes and is often **not meaningfully different** from therapist-delivered CBT‑I for many users, though adherence varies [20–22]. Some analyses report improvements such as **reduced sleep onset latency (~15 minutes)** and small increases in total sleep time in comparisons to inactive controls [20].  

**Practical advantage:** scalable, lower cost, can be done privately at home; best when paired with consistent sleep diary use.

---

## Mindfulness-based interventions (MBSR, workplace MBIs, internet mindfulness)

### Why mindfulness is relevant to work/study stress and rumination
Mindfulness training targets **attentional control**, **decentering** (seeing thoughts as events, not facts), and **reduced reactivity**, all of which map well to:  
- “My mind keeps running after work.”  
- “Thoughts keep me awake.”  
- “I can’t stop replaying conversations or tomorrow’s tasks.”

### Evidence highlights
- **Workplace mindfulness-based interventions** show substantial reductions in stress in some reviews (e.g., **stress SMD ~0.72**) and improvements in mental health/well-being outcomes [5].  
- **Internet mindfulness-based interventions (iMBIs)** show **small-to-moderate** benefits for stress (**g ~‑0.39**) and smaller effects for anxiety (**g ~‑0.20**) [4].  
- A systematic review of RCTs found mindfulness interventions reduce **ruminative thinking** (about **SMD ~‑0.53**) [6].

### Time & adherence realities
Classic MBSR is time-intensive (often 8 weeks with substantial home practice). Shorter formats and apps are easier to adopt but may yield smaller effects; consistency matters more than intensity for many people.

### When it may be less appropriate
If there is significant trauma history, severe dissociation, or acute mental health instability, unguided practice can sometimes be uncomfortable; a trauma-informed teacher/therapist can help titrate practice.

---

## Relaxation techniques (PMR, diaphragmatic breathing, guided imagery)

### Best role: “state change” after work + pre-sleep arousal reduction
Relaxation methods are particularly practical when the main problem is **physiological activation** (tension, racing heart, restlessness) and difficulty transitioning into rest.

### Evidence
- Studies comparing relaxation approaches show **guided imagery, diaphragmatic breathing, and PMR** can improve relaxation states versus controls [8].  
- PMR interventions have shown reductions in anxiety and improvements in sleep quality in some controlled settings (e.g., a 30-minute PMR intervention in healthcare practitioners) [9].

### Time-to-benefit
Often **same-day** subjective calming. For sleep improvement, typically **1–3 weeks** of regular practice is needed to see more stable change.

### Limitations
Relaxation can become counterproductive if used as a “performance task” (“I must relax now or I won’t sleep”). In that case, pairing relaxation with ACT-style acceptance or CBT‑I cognitive strategies often helps.

---

## Acceptance-based / ACT methods (including ACT‑I)

### Why ACT fits rumination and “can’t switch off”
ACT focuses on changing the relationship to thoughts (defusion, acceptance) and building a life guided by values even when stress is present. For bedtime, it can reduce the struggle with “not sleeping yet,” which itself perpetuates arousal.

### Evidence
Trials and pilot RCTs suggest ACT for insomnia (sometimes combined with behavioral elements like bedtime restriction) can improve insomnia severity and sleep-related quality of life, with some studies suggesting **no major difference vs CBT‑I** (though evidence base is smaller) [13–14].

### Practical note
ACT is relatively self-guidable (workbooks, courses), but many people benefit from a therapist/coach to apply it to real-world triggers (perfectionism, performance anxiety, workload boundaries).

---

## Exercise as a stress and sleep intervention

### What it helps most
- Lowers baseline stress reactivity, improves mood and energy regulation
- Improves subjective sleep quality, and sometimes sleep continuity over time

### Evidence highlights
- Reviews report improvements in sleep quality with **aerobic exercise** (e.g., around **SMD ~‑0.52** in one synthesis) [10].  
- Tai chi and other mind–body movement approaches show benefits for sleep and stress; a large RCT found tai chi might lag behind CBT‑I at 3 months but be **non-inferior by 12 months** for chronic insomnia outcomes in that study [12].  
- Broader reviews suggest movement-based mindfulness approaches can improve sleep quality [7], aligning with the idea that combining movement + attention regulation can be particularly helpful.

### Practical considerations
- For insomnia, avoid very intense workouts right before bed if it increases arousal; earlier day or late afternoon often works better (but individual responses vary).
- Consistency matters more than intensity extremes.

---

## Biofeedback (especially HRV biofeedback)

### Rationale
HRV biofeedback trains paced breathing and autonomic flexibility; it can be helpful when stress shows up strongly in the body (tight chest, agitation, palpitations) and for anxiety-like symptoms.

### Evidence
Meta-analytic and controlled-study literature suggests **promising** effects, with some syntheses reporting around **g ~0.38** for certain mental health outcomes and reported stress/anxiety improvements [18–19]. Evidence quality and protocols vary (device type, coaching, session frequency), which makes exact comparisons difficult.

### Practicality
Requires a device/app; some people do fine self-guided, others benefit from initial coaching to find their resonance breathing rate and prevent over-breathing.

---

## Journaling / expressive writing

### Best use-case
- “Brain dump” for worry loops
- Processing emotionally loaded work/study events
- Offloading tasks into a plan (especially if paired with time management)

### Evidence
A meta-analysis indicates **small but significant** improvements for anxiety, stress, and depressive symptoms, and benefits may be delayed rather than immediate [15].  

### Important timing note (for insomnia)
For people with bedtime rumination, journaling can help **if done earlier in the evening** (e.g., 1–3 hours before bed). Doing it right at bedtime can sometimes activate more thinking.

---

## Time-management / organizational interventions

### Why it matters for “near-daily” stress
If the stressor is chronic overload, the highest-yield intervention is often upstream: clarifying priorities, reducing task switching, setting boundaries, and building a realistic plan that prevents evening catch-up work.

### Evidence
Systematic reviews in education/work contexts suggest time management (planning, prioritization) is associated with better well-being and reduced stress, though study designs and measures vary [16–17]. Effects depend heavily on whether the intervention changes real constraints (deadlines, workload, interruptions) versus adding another “productivity demand.”

### Best practices (evidence-informed, low-risk)
- Weekly planning + daily “top 3” priorities
- Protected shutdown routine (see below)
- Limiting work in the last hour before bed (behavioral boundary for sleep)

---

## Which techniques to prioritize (ranked) for each of your three goals

## 1) Reduce near-daily work/study stress
**Highest-yield combination:**
- **Mindfulness-based program** (workplace MBI or structured course; digital if needed) for stress and rumination reduction [4–6].  
- **Time management / organizational changes** to reduce chronic triggers [16–17].  
- **Exercise** 3–5x/week for baseline resilience [10–12].  

**Fast add-on:** brief **paced breathing** (2–5 minutes) during the day, especially before/after stressful meetings or study blocks [8].

## 2) Relax after a long day
**Most practical stack:**
- **Transition ritual**: 10–20 minutes PMR or guided imagery + phone off [8–9].  
- **Light movement** (walk, gentle yoga/tai chi) if you carry stress somatically [12].  
- **“Shutdown complete” routine** (time-management boundary): write tomorrow’s first steps, capture loose ends, then stop work.

## 3) Nighttime rumination that keeps you awake (insomnia)
**Most directly supported approach:**
- **CBT‑I (or dCBT‑I)** as the core intervention [1–3,20–22].  
**Adjuncts that directly target rumination:**
- Mindfulness skills for decentering from thoughts [6].  
- ACT-style defusion/acceptance when you get stuck on “I must sleep” control struggles [13–14].  
- PMR/breathing to reduce arousal, but ideally not used as a “sleep performance test” [8–9].

---

## Practical implementation (open-ended assumptions + options)

Because age, health conditions, schedule, budget, and preference for apps vs in-person care weren’t specified, the most robust “default” plan is:

### Option A (self-guided, high evidence, insomnia-focused)
- **Start dCBT‑I** (6–8 weeks) if insomnia/rumination is the most impairing symptom and access to therapy is limited [20–22].  
- Add **10 minutes PMR** most evenings + **2–5 minutes paced breathing** during the day [8–9].  
- Add **light–moderate exercise** most days (even brisk walking) [10].

### Option B (stress + rumination-focused, if sleep issues are milder)
- **Structured mindfulness program** (8-week course or well-designed digital program) + **time management** changes [4–7,16–17].  
- If insomnia persists beyond ~4 weeks, step up to CBT‑I.

### Option C (therapist-supported, fastest tailoring)
- CBT‑I with a clinician if you have complex sleep patterns, comorbid mental health conditions, or repeated self-guided failures [1–3].  
- Consider ACT-informed therapy if rumination is strongly perfectionism/values conflict driven [13–14].

---

## Risks, contraindications, and “when to choose something else”

## When to seek clinical evaluation rather than self-manage only
- Suspected **sleep apnea** (snoring, gasping, daytime sleepiness), restless legs, parasomnias.
- Severe insomnia with major functional impairment, suicidal ideation, or severe depression/anxiety.
- Bipolar disorder history (sleep restriction can be destabilizing).

## Intervention-specific cautions
- **CBT‑I sleep restriction:** temporary sleepiness; requires caution with driving/safety tasks and certain medical/psychiatric conditions [2–3].  
- **Mindfulness:** may initially increase distress in some people; trauma-informed support can help [4–7].  
- **Breathing techniques:** avoid over-breathing; people prone to panic may need gentler pacing [8].  
- **Exercise:** injury/overtraining; very late high-intensity can worsen sleep for some [10].  
- **Expressive writing:** can transiently increase negative affect; avoid intense trauma processing alone [15].

---

## Key limitations in the evidence (and what that means for you)

- **Heterogeneity:** “Mindfulness,” “relaxation,” “exercise,” and “time management” cover many protocols; effect sizes vary by intensity, instructor quality, and adherence [4–7,10,16–17].  
- **Adherence drives outcomes:** CBT‑I works extremely well when implemented, but sleep restriction/stimulus control are behaviorally demanding; mindfulness benefits depend on practice time; exercise requires routine consistency.  
- **Comparability:** Different studies use different populations (students, workers, clinical insomnia), measures (stress scales vs ISI), and controls (waitlist vs active), which affects effect-size comparisons.  
- **Digital attrition:** Self-guided programs often have higher dropout; picking a program with reminders, accountability, or optional coaching can matter more than minor differences in content [20–22].

---

## Bottom line (most evidence-supported choices for your situation)
- For **nighttime rumination that keeps you awake**, **CBT‑I (including digital CBT‑I)** has the strongest, most direct evidence and guideline support as first-line insomnia treatment [1–3,20–22].  
- For **near-daily work/study stress and rumination**, **mindfulness-based interventions** (workplace or structured programs) have solid evidence for stress reduction and rumination decreases; internet versions help but tend to have smaller effects [4–6].  
- For **unwinding after the day**, **PMR + paced breathing/guided imagery** are the most practical rapid tools; they’re best used as consistent skills rather than “emergency fixes” [8–9].  
- For broad resilience and sleep support, **exercise** is a high-value, generally low-risk adjunct that compounds over time [10–12].  
- If rumination is driven by a strong “control battle” with thoughts and sleep, **ACT/ACT‑I** is a strong conceptual fit with promising evidence, though not as mature as CBT‑I [13–14].

---

### Sources
[1] Management of Chronic Insomnia Disorder in Adults (ACP Clinical Practice Guideline): https://www.acpjournals.org/doi/10.7326/M15-2175  
[2] Behavioral and Psychological Treatments for Chronic Insomnia Disorder in Adults (AASM Clinical Practice Guideline): https://jcsm.aasm.org/doi/10.5664/jcsm.8986  
[3] Network Meta-analysis of CBT-I Components (incl. sleep restriction findings): https://www.sciencedirect.com/science/article/pii/S0272735824001284  
[4] Internet Mindfulness-Based Interventions for Stress Management (Meta-analysis): https://pmc.ncbi.nlm.nih.gov/articles/PMC7360300/  
[5] Workplace Mindfulness-Based Interventions and Employee Outcomes (Systematic Review/Meta-analysis): https://pmc.ncbi.nlm.nih.gov/articles/PMC7360300/  
[6] Mindfulness-Based Interventions for Ruminative Thinking (Systematic Review of RCTs): https://www.sciencedirect.com/science/article/abs/pii/S0165032722012137  
[7] Mindfulness-Based Movement Intervention to Improve Sleep Quality (Systematic Review/Meta-analysis): https://www.mdpi.com/1660-4601/19/16/10284  
[8] Effectiveness of Progressive Muscle Relaxation, Deep Breathing, and Guided Imagery (Comparative study): https://www.semanticscholar.org/paper/Effectiveness-of-Progressive-Muscle-Relaxation%2C-and-Toussaint-Nguyen/a2bc2984fa329911d816c06f6c165a571b48e2b9  
[9] Progressive Muscle Relaxation Alleviates Anxiety and Improves Sleep Quality (Study): https://www.frontiersin.org/journals/psychology/articles/10.3389/fpsyg.2024.1337318/full  
[10] Effects of Aerobic Exercises at Different Intensities on Sleep Quality (Review): https://www.dovepress.com/effects-of-aerobic-exercises-at-different-intensities-on-sleep-quality-peer-reviewed-fulltext-article-NSS  
[11] Exercise Modalities and Sleep (Systematic review; Pilates noted as effective): https://link.springer.com/article/10.1186/s12889-025-22570-1  
[12] Tai Chi versus Cognitive Behavioral Therapy for Chronic Insomnia (RCT): https://www.bmj.com/content/391/bmj-2025-084320  
[13] Acceptance and Commitment Therapy Combined with Bedtime Restriction vs CBT for Insomnia (Pilot RCT): https://www.researchgate.net/publication/378570995_Acceptance_and_Commitment_Therapy_combined_with_bedtime_restriction_versus_Cognitive_Behavioral_Therapy_for_insomnia_a_randomized_controlled_pilot_trial  
[14] Acceptance and Commitment Therapy for Nurses’ Sleep Quality (Study): https://www.elsevier.es/en-revista-international-journal-clinical-health-psychology-355-articulo-acceptance-commitment-therapy-for-nurses-S1697260025000018  
[15] Expressive Writing and Mental Health Outcomes (Meta-analysis): https://pubmed.ncbi.nlm.nih.gov/36536513/  
[16] Boosting Productivity and Wellbeing Through Time Management (Systematic Review): https://www.frontiersin.org/journals/education/articles/10.3389/feduc.2025.1623228/full  
[17] Effects of Time Management Intervention on Mental Health and Wellbeing (Preprint): http://medrxiv.org/cgi/reprint/2023.07.07.23292349v1  
[18] Efficacy of HRV Biofeedback and Depressive Symptoms (Systematic Review/Meta-analysis): https://www.nature.com/articles/s41598-021-86149-7  
[19] The Effect of HRV Biofeedback Training on Stress and Anxiety (Meta-analysis): https://www.researchgate.net/publication/316802722_The_effect_of_heart_rate_variability_biofeedback_training_on_stress_and_anxiety_a_meta-analysis  
[20] Effects of Digital Cognitive Behavioral Therapy for Insomnia on Self-Reported Sleep Outcomes (Review): https://pmc.ncbi.nlm.nih.gov/articles/PMC12731755/  
[21] Digital Cognitive Behavioral Therapy for Insomnia Using a Smartphone (RCT): https://jamanetwork.com/journals/jamanetworkopen/fullarticle/2802765  
[22] Efficacy of Digital Cognitive Behavioral Therapy for the Treatment of Insomnia (Trial/Review): https://jamanetwork.com/journals/jamapsychiatry/fullarticle/2758827


Research workflow completed!
